In [13]:
import numpy as np
from numpy.random import randint
import pandas as pd

In [14]:
meta = pd.read_csv("./SimulatedData_SampleSize100.csv")
del meta['Unnamed: 0']
meta_name = pd.read_csv("./id_name.txt", sep=':', header = None)
meta.columns = meta_name.values[:,-1]

In [15]:
meta.head()

,DLI C1-C2,DLI C2-C3,DLI C3-C4,DLI C4-C5,DLI C5-C6,DLI C6-C7,DLI C7-C8,DLI C8-T1,DLI L1-L2,DLI L2-L3,...,Breast backache,Chest discomfort,L breast problems,R breast problems,Toracal dysfunction,Upper abdominal discomfort,Lateral abdominal discomfort,Abdominal discomfort,L lower abdominal discomfort,Lower abdominal discomfort
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
meta.shape

(100, 222)

In [17]:
Symptom = ' R shoulder impingement '
pos_indeces = np.arange(len(meta[Symptom].values))[meta[Symptom].values>0]
neg_indeces = np.arange(len(meta[Symptom].values))[meta[Symptom].values==0]
#neg_indeces = np.random.choice(neg_indeces, len(pos_indeces), replace=False)

In [18]:
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.GraphUtils import GraphUtils

cols = [' R shoulder impingement ', ' R C4 Radiculopathy ', ' R C5 Radiculopathy ', ' R C6 Radiculopathy ']#, ' DLI C3-C4 ', ' DLI C4-C5 ',  ' DLI C5-C6 ']

# Causal Discovery (FCI)
# g, edges = fci(meta[cols].values[np.concatenate([pos_indeces, neg_indeces]),:].astype(float), alpha = 0.5, verbose=False)
g, edges = fci(meta[cols].values.astype(float), alpha = 0.5, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=cols)
print(pdy.to_string())

/opt/miniconda3/envs/cirl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Depth=0, working on node 3: 100%|██████████| 4/4 [00:00<00:00, 2053.51it/s]

X4 --> X1
digraph {
fontsize=18;
dpi=200;
0 [label=X1];
0 [label=" R shoulder impingement "];
1 [label=X2];
1 [label=" R C4 Radiculopathy "];
2 [label=X3];
2 [label=" R C5 Radiculopathy "];
3 [label=X4];
3 [label=" R C6 Radiculopathy "];
1 -> 0 [dir=both, arrowtail=none, arrowhead=normal];
3 -> 0 [dir=both, arrowtail=none, arrowhead=normal];
1 -> 3 [dir=both, arrowtail=odot, arrowhead=normal];
2 -> 3 [dir=both, arrowtail=odot, arrowhead=normal];
}



In [19]:
get_symptom = lambda idx: [sym.strip() for sym in meta.iloc[idx,-143:][meta.iloc[idx,-143:]>0].index]
get_pattern  = lambda idx:  [sym.strip() for sym in meta.iloc[idx,26:-143][meta.iloc[idx,26:-143]>0].index]
get_pathophysiological  = lambda idx:  [sym.strip() for sym in meta.iloc[idx,:26][meta.iloc[idx,:26]>0].index]

In [20]:
get_pattern(45)

['L C5 Radiculopathy',
 'R C5 Radiculopathy',
 'L T10 Radiculopathy',
 'R T10 Radiculopathy',
 'L S1 Radiculopathy']

In [21]:
get_pathophysiological(45)

['DLI C4-C5', 'DLI L5-S1', 'DLI T9-T10']

In [22]:
get_gender = lambda: 'Male' if np.random.uniform() > 0.5 else 'Female'

In [23]:
def neuro_prompt(idx):
    symptom_notes = '\n- '.join(['']+get_symptom(idx))
    pattern_notes = '\n- '.join(['']+get_pattern(idx))
    pathophysiological_notes = '\n- '.join(['']+get_pathophysiological(idx))
    prompt = f'''
You are a helpful and professional assistant to Neurologists. Try your best!

Write a clinical notes based on following informaiton:

Patient id: Neuro_{idx:05}
Age: {10+np.random.choice(50)}
Gender: {get_gender()}

---

Write one summary (one paragraph, do not do inference) about **symptom diagnosis**:
{symptom_notes}

Write one summary (one paragraph, do not do inference) about **pattern diagnosis**:
{pattern_notes}

Write one summary (one paragraph, do not do inference) about **pathophysiological diagnosis**:
{pathophysiological_notes}

---
L: Left
R: Right
DLI: discoligamentous injury
Radi: radiculopathy 

---

Your clinical notes should surrounded by <note> ...  </note> 

''' 
    return prompt

In [59]:


data_file = "./data.txt"
with open(data_file, 'r') as file:
    data_items = file.read().split('---')[1:]
    data_items = [item.split("\n\n",1)[1] for item in data_items]

In [60]:
data_items[4]

'**R shoulder impingement**: 0\n**Notes**:\n\nPatient ID: Neuro_00004\nAge: 25\nGender: Male\n\nSymptom Diagnosis:\nThe patient presented with right side ptosis (R pta), lower back pain (lumbago), pain in the left hip joint, left heel issues, pain in the left rear thigh, and discomfort consistent with left side Achilles tendon pain (L achillodyni). These symptoms require further evaluation to determine the underlying causes and appropriate management strategies.\n\nPattern Diagnosis:\nThe diagnostic patterns identified include left side L5 radiculopathy, right side L5 radiculopathy, and left side S1 radiculopathy. These patterns suggest that the patient may be experiencing symptoms related to nerve root irritation or compression at specific levels of the lumbar spine.\n\nPathophysiological Diagnosis:\nThe patient has been diagnosed with discoligamentous injuries at the L4-L5 and L5-S1 levels of the spine (DLI L4-L5 and DLI L5-S1). These injuries may contribute to the symptomatology and

In [25]:
data = {
    'Notes': [],
    'R shoulder impingement': [],
}
sample_size = 0
additional_sample_number = 100

for idx in range(additional_sample_number):

    
    label =  meta[' R shoulder impingement '].values[idx]

    data['Notes'].append(data_items[idx])
    data['R shoulder impingement'].append(label)

    sample_size += 1




In [26]:
df_data = pd.DataFrame(data)
add_cols = [' R C4 Radiculopathy ', ' R C5 Radiculopathy ', ' R C6 Radiculopathy ']
for each_col in add_cols:
    df_data[each_col.strip()] = meta[each_col]

In [27]:
pd.DataFrame(data).to_csv("./neuro_R_shoulder_impingement.csv", index=False) 

In [28]:
df_data.head()

,Notes,R shoulder impingement,R C4 Radiculopathy,R C5 Radiculopathy,R C6 Radiculopathy
0,**R shoulder impingement**: 0\n**Notes**:\n\nP...,0,0,0,1
1,Patient ID: Neuro_00001\nAge: 19\nGender: Male...,0,0,0,0
2,Patient ID: Neuro_00002\nAge: 46\nGender: Male...,0,0,0,0
3,Patient ID: Neuro_00003\nAge: 44\nGender: Male...,1,0,0,0
4,Patient ID: Neuro_00004\nAge: 25\nGender: Male...,0,0,0,0


In [187]:
score_set

array([0, 1])

"\nPatient ID: Neuro_00001\nAge: 19\nGender: Male\n\nSummary of Symptom Diagnosis:\nThe patient presents with pain in the neck region, accompanied by left shoulder impingement which is indicative of limitations in normal shoulder mobility and function. The patient reports issues specifically with the left shoulder, including non-specific troubles and problems that affect shoulder movement and stability. Additionally, the right shoulder is mentioned to have troubles, although further details on the right shoulder issues are not provided. The combination of these symptoms suggests a multifocal involvement of the shoulder girdle with a predominance on the left side.\n\nSummary of Pattern Diagnosis:\nThe patient demonstrates symptoms consistent with left C4 radiculopathy. This pattern of diagnosis suggests an involvement of the nerve root emanating from the spinal cord at the level of the cervical vertebrae C4, which could explain the symptomatology affecting the patient’s neck and left sh

In [194]:
import pandas as pd
pd_data = df_data
score_set = pd_data['R shoulder impingement'].unique()
score_set = score_set[np.argsort(score_set)]

sample_size = pd_data.shape[0]

sorted_data = '# Data\n'

for v in score_set:

    sorted_data += f'## Group with \'R shoulder impingement\'={v}\n\n'
    
    this_indeces = np.arange(sample_size)[pd_data['R shoulder impingement'].values == v]
    for each_idx in this_indeces:
        sorted_data += f'----\n\n{pd_data["Notes"].values[each_idx]}\n\n'


print(sorted_data)


# Data
## Group with 'R shoulder impingement'=0

----


Patient ID: Neuro_00000
Age: 57
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a left shoulder impingement and associated intracapsular problems. The symptoms include limited and painful range of motion with a positive outcome when performing a "thumbs up" test on the left side. Right upper limb assessment also reveals issues; the patient wears an armband suggesting ongoing management for right lower arm disorders. The patient reports lumbago and pain localized to the left thigh. Physical therapy assessment (L pta) is noted, indicating involvement in a therapeutic intervention. Joint assessments reveal a left hip joint condition and right hip arthritis, accompanied by right hamstring discomfort.

Pattern Diagnosis Summary:
Neurological examination indicates pattern diagnoses of bilateral C6 radiculopathy and bilateral L5 radiculopathy. The bilateral presentation suggests nerve root compression or irritation at 